In [1]:
import numpy as np
import pandas as pd
import random

#### Valores Globais

In [2]:
states   = 6       # número de estados (grid 2x3)
actions  = 5       # número de ações   (cima, baixo, esquerda, direita) + loop

row      = 2       # número de linhas do grid
col      = 3       # número de colunas do grid

lr       = 0.1     # taxa de aprendizado (learning rate)
gamma    = 0.9     # taxa de desconto (gamma)

goal     = (0, 2)  # posição final
iteractions = 65   # número de iterações do agente

#### Classe Estado 

In [3]:
class q_state():
    
    # construtor
    def __init__(self, position):
        self.id = -1
        self.goal = False
        self.actions = {}
        self.position = position
    
    # método para atribuir id da coluna da tabela Q
    def get_id(self):
        
        if (self.position == (0,0)):
            self.id = 0
            
        elif (self.position == (0,1)):
            self.id = 1
            
        elif (self.position == (0,2)):
            self.id = 2
            
        elif (self.position == (1,0)):
            self.id = 3
            
        elif (self.position == (1,1)):
            self.id = 4
            
        elif (self.position == (1,2)):
            self.id = 5

        return self.id
    
    # método para atribuir as ações possíveis de um estado
    def get_actions(self):
        
        self.id = self.get_id()
        
        if (self.id == 0):
            self.actions = {"right" : 3, "down" : 1}
        
        elif (self.id == 1):
            self.actions = {"left" : 2, "right" : 3, "down" : 1}
            
        elif (self.id == 2):
            self.actions = {"loop" : 4}
            
        elif (self.id == 3):
            self.actions = {"up" : 0, "right" : 3}
            
        elif (self.id == 4):
            self.actions = {"left" : 2, "right" : 3, "up" : 0}
            
        elif (self.id == 5):
            self.actions = {"up" : 0, "left" : 2}
            
        return self.actions
    
    # método para verificar se a posição do estado é o objetivo
    def is_goal(self):
        
        if (self.position == goal):
            self.goal = True
            
        else:
            self.goal = False
            
        return self.goal
    
    # método para atribuir recompensa para o estado
    def get_reward(self):
        
        if (self.position == goal):
            return 100
            
        else:
            return 0

#### Classe Agente

In [4]:
class q_agent():
    
    # construtor
    def __init__(self):
        
        self.Q = np.zeros((states, actions))
        self.iteractions = iteractions
        
    # método para iniciar posição para treinamento
    def random_init(self):
        
        x = goal[0]
        y = goal[1]
        
        while (x == goal[0] and y == goal[1]):
            
            x = random.randint(0, row - 1)
            y = random.randint(0, col - 1)
        
        return x, y
    
    # método para retornar as posições do novo estado a partir de uma ação
    def next_state(self, action, x, y):
        
        # cima
        if (action == 0):
            x = x - 1
        
        # baixo
        elif (action == 1):
            x = x + 1
        
        # esquerda
        elif (action == 2):
            y = y - 1
        
        # direita
        elif (action == 3):
            y = y + 1
            
        return q_state((x, y))
        
    # atualiza tabela Q
    def update_q_table(self, Q, s, a, r, ns, na):
        
        self.Q[s, a] = self.Q[s, a] + lr * (r + gamma * self.Q[ns, na] - self.Q[s, a])
        
    def run(self):
        
        goal = False
        
        for i in range(self.iteractions):
            
            # seleciona um valor x e y como posições iniciais
            x, y = self.random_init()
            
            # cria objeto de estado
            st = q_state((x,y))
            
            while not st.is_goal():
                               
                # escolhe uma ação do estado aleatoriamente
                action = random.choice(list(st.get_actions().values()))
                
                # cria um objeto com o próximo estado da ação selecionada
                next_st = self.next_state(action, x, y)
                
                # verifica a recompensa do estado escolhido
                r = next_st.get_reward()
                
                # seleciona uma ação aleatória para o próximo estado        
                next_action = random.choice(list(next_st.get_actions().values()))
                
                # atualiza tabela Q
                self.update_q_table(self.Q, st.get_id(), action, r, next_st.get_id(), next_action)
                
                # o estado atual é o próximo estado escolhido
                st = next_st
                
                # atualiza os novos valores de x e y
                x, y = next_st.position

#### Métodos Auxiliares para Tabelas do Q-Learning 

In [5]:
def generate_q_table(Q):

    columns = ["up", "down", "left", "right", "loop"]
    
    df = pd.DataFrame(data = Q, 
                      index = ["s" + str(i + 1) for i in range(Q.shape[0])], 
                      columns = columns)
    
    df = df.round(2)
    
    print('\033[1m\tQ Table\033[0m\n')
    print(df)
    
def generate_v_table(Q):
    
    s1_max = np.amax(Q[0], axis=0)
    s2_max = np.amax(Q[1], axis=0)
    s3_max = np.amax(Q[2], axis=0)
    s4_max = np.amax(Q[3], axis=0)
    s5_max = np.amax(Q[4], axis=0)
    s6_max = np.amax(Q[5], axis=0)
        
    print('\033[1m\n\tV Table\033[0m\n')
    print('\t----------------')
    print('\t| %d | %d | %d  |' % (s1_max, s2_max, s3_max))
    print('\t----------------')
    print('\t| %d | %d | %d |' % (s4_max, s5_max, s6_max))
    print('\t----------------\n')
    
def generate_policy_table(Q):
    
    s1 = action_char(np.argmax(Q[0], axis=0))
    s2 = action_char(np.argmax(Q[1], axis=0))
    s3 = action_char(np.argmax(Q[2], axis=0))
    s4 = action_char(np.argmax(Q[3], axis=0))
    s5 = action_char(np.argmax(Q[4], axis=0))
    s6 = action_char(np.argmax(Q[5], axis=0))
        
    print('\033[1m\n\tPolicy Table\033[0m\n')
    print('\t----------------')
    print('\t|  %s |  %s |  %s |' % (s1, s2, s3))
    print('\t----------------')
    print('\t|  %s |  %s |  %s |' % (s4, s5, s6))
    print('\t----------------\n')

def action_char(index):
    
    if (index == 0):
        return '^'
        
    elif (index == 1):
        return 'v'
        
    elif (index == 2):
        return '<'
        
    elif (index == 3):
        return '>'
        
    elif (index == 4):
        return '*'

#### Função Principal 

In [6]:
def main():
    
    print('\033[1m\n\tGridworld\033[0m\n')
    print('\t----------------\t\t----------------------------')
    print('\t| s1 | s2 | s3 |\t\t| (0, 0) | (0, 1) | (0, 2) |')
    print('\t----------------\t\t----------------------------')
    print('\t| s4 | s5 | s6 |\t\t| (1, 0) | (1, 1) | (1, 2) |')
    print('\t----------------\t\t----------------------------\n')
    
    # cria objeto para o agente
    qa = q_agent()
    
    # executa o agente
    qa.run()
    
    Q = qa.Q

    # gera tabela Q*
    generate_q_table(Q)
    
    # gera tabela V*
    generate_v_table(Q)
    
    # imprime política ótima
    generate_policy_table(Q)
    
if __name__ == "__main__": main()


	Gridworld

	----------------		----------------------------
	| s1 | s2 | s3 |		| (0, 0) | (0, 1) | (0, 2) |
	----------------		----------------------------
	| s4 | s5 | s6 |		| (1, 0) | (1, 1) | (1, 2) |
	----------------		----------------------------

	Q Table

       up   down   left  right  loop
s1   0.00  26.13   0.00  49.38   0.0
s2   0.00  24.51  32.63  97.50   0.0
s3   0.00   0.00   0.00   0.00   0.0
s4  29.31   0.00   0.00  33.12   0.0
s5  45.85   0.00  21.14  53.98   0.0
s6  95.76   0.00  24.22   0.00   0.0

	V Table

	----------------
	| 49 | 97 | 0  |
	----------------
	| 33 | 53 | 95 |
	----------------


	Policy Table

	----------------
	|  > |  > |  ^ |
	----------------
	|  > |  > |  ^ |
	----------------

